# Final Project Submission
- Student Name - Vinayak Modgil
- Student pace - Full time
- Scheduled project review time/date -
- Instructor name - Yish Lim
- Plog Post URL - 

# Table of Contents
- [Introduction](#Introduction)
- [Data Collection](#Data-Collection)
- [Data Cleaning](#Data-Cleaning)
- [Data Exploration](#Data-Exploration)
- [Data Modeling](#Data-Modeling)
- [Data Interpretation](#Data-Interpretation)
- [Recommendations and Conclusions](#Recommendations-and-Conclusions)

# Introduction

# Data Collection

## Import necessary packages

In [84]:
import pandas as pd
import os, sys, glob
import cv2

import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from PIL import Image

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout, Activation
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


## Global Functions

In [85]:
def n_images_identifier(base_folder="data/chest_xray/chest_xray/"):
    '''
    returns the number of different
    types of images in the folders
    
    n_images_identifies: 
    - base folder : common folder holding all the images.
    '''
    #Takes the folder type as normal or pneumonia
    folder_type = ["NORMAL", "PNEUMONIA"]
    #
    train_folder = base_folder + "train/"
    test_folder = base_folder + "test/"
    val_folder = base_folder + "val/"
    
    # iterates for folder in train_folder, test_folder
    #and val_folder
    for folder in [train_folder, test_folder, val_folder]:
        
        #Further iterates for _type in folder_type
        for _type in folder_type:
            if _type=="NORMAL":
                n_normal = len(os.listdir(folder + _type))
            else:
                n_infected = len(os.listdir(folder + _type))
        print(f"Set: {folder}, normal images: {n_normal}, infected images: {n_infect}")
    return 
    

In [94]:
def data_processing(img_dims,
                    batch_size,
                    base_folder="data/chest_xray/chest_xray/"):
    train_folder = base_folder + "train/"
    test_folder = base_folder + "test/"
    val_folder = base_folder + "val/"
    #Data genereation objects
    train_images = ImageDataGenerator(rescale=1/255)
    test_val = ImageDataGenerator(rescale=1/255)
    
    
    #Use flow from directory to get train and test 
    #images
    train_chest = train_images.flow_from_directory(
    directory=train_folder,
    target_size=(img_dims, img_dims),
    batch_size=batch_size, 
    class_mode="binary", 
    shuffle=True)
    
    test_chest = test_val.flow_from_directory(
    directory=test_folder, 
    target_size=(img_dims, img_dims),
    batch_size=batch_size, 
    class_mode="binary", 
    shuffle=True)
    
    # Getting labels and data separated
    
    test_data = []
    test_labels = []
    for folder in ["NORMAL/", "PNEUMONIA/"]:
        for img in os.listdir(test_folder + folder):
            img = plt.imread(test_folder+ folder+img)
            img = cv2.resize(img, (img_dims, img_dims))
            img = np.dstack([img, img, img])
            img = img.astype("float32") / 255
            if folder == "NORMAL/":
                label = 0
            elif folder =="PNEUMONIA/":
                label=1
            test_data.append(img)
            test_labels.append(label)
    
    test_data = np.array(test_data)
    test_labels = np.array(test_labels)
    
    return train_chest, test_chest, test_data, test_labels

In [95]:
data_processing(32, 100)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


(<tensorflow.python.keras.preprocessing.image.DirectoryIterator at 0x1d71bbe07c8>,
 array([[[[0.11764706, 0.11764706, 0.11764706],
          [0.10588235, 0.10588235, 0.10588235],
          [0.11372549, 0.11372549, 0.11372549],
          ...,
          [0.11372549, 0.11372549, 0.11372549],
          [0.11764706, 0.11764706, 0.11764706],
          [0.1254902 , 0.1254902 , 0.1254902 ]],
 
         [[0.11372549, 0.11372549, 0.11372549],
          [0.10588235, 0.10588235, 0.10588235],
          [0.09411765, 0.09411765, 0.09411765],
          ...,
          [0.10196079, 0.10196079, 0.10196079],
          [0.10588235, 0.10588235, 0.10588235],
          [0.13725491, 0.13725491, 0.13725491]],
 
         [[0.11764706, 0.11764706, 0.11764706],
          [0.09803922, 0.09803922, 0.09803922],
          [0.07843138, 0.07843138, 0.07843138],
          ...,
          [0.09019608, 0.09019608, 0.09019608],
          [0.09411765, 0.09411765, 0.09411765],
          [0.10588235, 0.10588235, 0.10588235]],
 